# 경사 하강법을 이용한 얕은 신경망 학습


In [11]:
import tensorflow as tf
import numpy as np

## 하이퍼 파라미터 설정

In [12]:
EPOCHS = 1000

## 네트워크 구조 정의
### 얕은 신경망
#### 입력 계층 : 2, 은닉 계층 : 128 (Sigmoid activation), 출력 계층 : 10 (Softmax activation)

In [26]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.d1 = tf.keras.layers.Dense(128, input_dim=2, activation = 'sigmoid' ) # Dense Layer == Fully Connected Layer
        self.d2 = tf.keras.layers.Dense(10, activation = 'softmax' )
        
    def call(self, x, training=None, mask=None):
        x = self.d1(x)
        return self.d2(x)

## 학습 루프 정의

In [29]:
@tf.function # Auto-graph라는 방법을 통해서, tf 최적화 진행된다고 함...4-06강 6분 36초
def train_step(model, inputs, labels, loss_object, optimizer, tarin_loss, train_metric):
    with tf.GradientTape() as tape: # 안의 값들에 대한 Gradients 계산돼서 tape안에 들어감
        predictions = model(inputs)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables) # loss를 model.trainable_variables로 편미분함, d f(x)/d x
    
    optimizer.apply_gradients( zip(gradients, model.trainable_variables) )
    train_loss(loss)
    train_metric(labels, predictions)

In [30]:
for a in zip([1,2,3,4,5],[2,3,4,5,6]):
    print(a)

(1, 2)
(2, 3)
(3, 4)
(4, 5)
(5, 6)


## 데이터셋 생성, 전처리

In [31]:
np.random.seed(0)

pts = list()
labels = list()

# 점 10개를 찍어서, Gaussian Distribution으로 각각 100개씩 만들거임.

center_pts = np.random.uniform(-8.0, 8.0, (10, 2))
for label, center_pt in enumerate(center_pts):
    for _ in range(100):
        pts.append(center_pt + np.random.randn(*center_pt.shape)) # 맨뒤 argument뭐지
        labels.append(label)
        
pts = np.stack(pts, axis=0).astype(np.float32) # GPU로 계산할 때는, float32로 해야함. 
# np.stack : numpy object로 바꾸는 함수, 여기선 LIst => numpy로 바꿈
labels = np.stack(labels, axis=0)

train_ds = tf.data.Dataset.from_tensor_slices( (pts, labels) ).shuffle(1000).batch(32) # Train Dataset으로 바꿔주는 역할
# 여기서는 Numpy 를 넣었지만, TENSOR를 넣어도 작동함


## 모델 생성

In [32]:
model = MyModel()

## 손실 함수 및 최적화 알고리즘 설정
### CrossEntropy, Adam Optimizer

In [33]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy() # Cross Entropy Error, 
# Categorical : classification에 사용됨 의미
# Sparse : One-Hot Encoding에서 나온 확률들 중 가장 높은 확률을 가지는 부분의 index를 반환한다는 뜻.

optimizer = tf.keras.optimizers.Adam()

## 평가 지표 설정
### Accuracy

In [34]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')


## 학습 루프

In [36]:
for epoch in range(EPOCHS):
    for x, label in train_ds:
        train_step(model, x, label, loss_object, optimizer, train_loss, train_accuracy)
        
        template = 'Epoch: {}, Loss: {}, Accuracy: {}'
    print(template.format(epoch + 1,
                         train_loss.result(),
                         train_accuracy.result() * 100))

Epoch: 1, Loss: 0.29088887572288513, Accuracy: 88.87567138671875
Epoch: 2, Loss: 0.2908405065536499, Accuracy: 88.87647247314453
Epoch: 3, Loss: 0.29078346490859985, Accuracy: 88.87752532958984
Epoch: 4, Loss: 0.2907373607158661, Accuracy: 88.87870025634766
Epoch: 5, Loss: 0.2906854450702667, Accuracy: 88.87999725341797
Epoch: 6, Loss: 0.2906361222267151, Accuracy: 88.88103485107422
Epoch: 7, Loss: 0.2905768156051636, Accuracy: 88.8822021484375
Epoch: 8, Loss: 0.29053258895874023, Accuracy: 88.88361358642578
Epoch: 9, Loss: 0.29048070311546326, Accuracy: 88.884521484375
Epoch: 10, Loss: 0.290422648191452, Accuracy: 88.88580322265625
Epoch: 11, Loss: 0.290365070104599, Accuracy: 88.88733673095703
Epoch: 12, Loss: 0.2903064787387848, Accuracy: 88.88861083984375
Epoch: 13, Loss: 0.2902526259422302, Accuracy: 88.88987731933594
Epoch: 14, Loss: 0.29020220041275024, Accuracy: 88.8905258178711
Epoch: 15, Loss: 0.29014402627944946, Accuracy: 88.89141082763672
Epoch: 16, Loss: 0.290089070796966

Epoch: 128, Loss: 0.2848859131336212, Accuracy: 89.00456237792969
Epoch: 129, Loss: 0.2848460376262665, Accuracy: 89.00565338134766
Epoch: 130, Loss: 0.2848067283630371, Accuracy: 89.00641632080078
Epoch: 131, Loss: 0.2847626805305481, Accuracy: 89.00761413574219
Epoch: 132, Loss: 0.28472185134887695, Accuracy: 89.00836944580078
Epoch: 133, Loss: 0.2846868932247162, Accuracy: 89.00967407226562
Epoch: 134, Loss: 0.28464755415916443, Accuracy: 89.01020812988281
Epoch: 135, Loss: 0.28460296988487244, Accuracy: 89.01139831542969
Epoch: 136, Loss: 0.2845563590526581, Accuracy: 89.0124740600586
Epoch: 137, Loss: 0.28452828526496887, Accuracy: 89.01398468017578
Epoch: 138, Loss: 0.28448641300201416, Accuracy: 89.01473236083984
Epoch: 139, Loss: 0.2844401001930237, Accuracy: 89.01590728759766
Epoch: 140, Loss: 0.2844032347202301, Accuracy: 89.0168685913086
Epoch: 141, Loss: 0.28436851501464844, Accuracy: 89.01815032958984
Epoch: 142, Loss: 0.2843257784843445, Accuracy: 89.01899719238281
Epoch:

Epoch: 257, Loss: 0.28015851974487305, Accuracy: 89.12891387939453
Epoch: 258, Loss: 0.28012654185295105, Accuracy: 89.1292724609375
Epoch: 259, Loss: 0.28010350465774536, Accuracy: 89.1304931640625
Epoch: 260, Loss: 0.28007054328918457, Accuracy: 89.13133239746094
Epoch: 261, Loss: 0.2800348401069641, Accuracy: 89.13245391845703
Epoch: 262, Loss: 0.2800024151802063, Accuracy: 89.13289642333984
Epoch: 263, Loss: 0.279964417219162, Accuracy: 89.13420867919922
Epoch: 264, Loss: 0.2799261212348938, Accuracy: 89.1351318359375
Epoch: 265, Loss: 0.27989137172698975, Accuracy: 89.13633728027344
Epoch: 266, Loss: 0.2798599600791931, Accuracy: 89.1375503540039
Epoch: 267, Loss: 0.27982521057128906, Accuracy: 89.1386489868164
Epoch: 268, Loss: 0.2797892391681671, Accuracy: 89.1392822265625
Epoch: 269, Loss: 0.27975207567214966, Accuracy: 89.14067077636719
Epoch: 270, Loss: 0.2797219157218933, Accuracy: 89.14177703857422
Epoch: 271, Loss: 0.2796909511089325, Accuracy: 89.14230346679688
Epoch: 272

Epoch: 384, Loss: 0.27628856897354126, Accuracy: 89.24540710449219
Epoch: 385, Loss: 0.2762722373008728, Accuracy: 89.24562072753906
Epoch: 386, Loss: 0.2762507498264313, Accuracy: 89.24627685546875
Epoch: 387, Loss: 0.2762189507484436, Accuracy: 89.24674224853516
Epoch: 388, Loss: 0.276187539100647, Accuracy: 89.24764251708984
Epoch: 389, Loss: 0.2761578857898712, Accuracy: 89.24871826171875
Epoch: 390, Loss: 0.2761269509792328, Accuracy: 89.24969482421875
Epoch: 391, Loss: 0.27610552310943604, Accuracy: 89.25076293945312
Epoch: 392, Loss: 0.2760739326477051, Accuracy: 89.25173950195312
Epoch: 393, Loss: 0.2760547995567322, Accuracy: 89.25271606445312
Epoch: 394, Loss: 0.27602848410606384, Accuracy: 89.2534408569336
Epoch: 395, Loss: 0.27599984407424927, Accuracy: 89.25415802001953
Epoch: 396, Loss: 0.2759709358215332, Accuracy: 89.25521087646484
Epoch: 397, Loss: 0.275942862033844, Accuracy: 89.25593566894531
Epoch: 398, Loss: 0.27591726183891296, Accuracy: 89.25689697265625
Epoch: 3

Epoch: 515, Loss: 0.2729550004005432, Accuracy: 89.3506088256836
Epoch: 516, Loss: 0.2729265093803406, Accuracy: 89.35126495361328
Epoch: 517, Loss: 0.27290335297584534, Accuracy: 89.35191345214844
Epoch: 518, Loss: 0.27287721633911133, Accuracy: 89.35279846191406
Epoch: 519, Loss: 0.2728530466556549, Accuracy: 89.35398864746094
Epoch: 520, Loss: 0.27282941341400146, Accuracy: 89.35456085205078
Epoch: 521, Loss: 0.27280256152153015, Accuracy: 89.35505676269531
Epoch: 522, Loss: 0.27278822660446167, Accuracy: 89.35624694824219
Epoch: 523, Loss: 0.272762656211853, Accuracy: 89.35665893554688
Epoch: 524, Loss: 0.2727483808994293, Accuracy: 89.35722351074219
Epoch: 525, Loss: 0.27272099256515503, Accuracy: 89.35787200927734
Epoch: 526, Loss: 0.2727005183696747, Accuracy: 89.35874938964844
Epoch: 527, Loss: 0.2726764678955078, Accuracy: 89.35954284667969
Epoch: 528, Loss: 0.27265313267707825, Accuracy: 89.36041259765625
Epoch: 529, Loss: 0.2726302742958069, Accuracy: 89.36090087890625
Epoch

Epoch: 642, Loss: 0.27016913890838623, Accuracy: 89.4375228881836
Epoch: 643, Loss: 0.27014675736427307, Accuracy: 89.43798828125
Epoch: 644, Loss: 0.2701267898082733, Accuracy: 89.43880462646484
Epoch: 645, Loss: 0.2701070010662079, Accuracy: 89.4395523071289
Epoch: 646, Loss: 0.2700849771499634, Accuracy: 89.44001007080078
Epoch: 647, Loss: 0.2700660824775696, Accuracy: 89.44047546386719
Epoch: 648, Loss: 0.27004274725914, Accuracy: 89.4412841796875
Epoch: 649, Loss: 0.2700241208076477, Accuracy: 89.44202423095703
Epoch: 650, Loss: 0.269999623298645, Accuracy: 89.44297790527344
Epoch: 651, Loss: 0.26997917890548706, Accuracy: 89.44392395019531
Epoch: 652, Loss: 0.2699560821056366, Accuracy: 89.44437408447266
Epoch: 653, Loss: 0.2699343264102936, Accuracy: 89.4450454711914
Epoch: 654, Loss: 0.2699139416217804, Accuracy: 89.44522094726562
Epoch: 655, Loss: 0.2698957920074463, Accuracy: 89.44574737548828
Epoch: 656, Loss: 0.2698749601840973, Accuracy: 89.44634246826172
Epoch: 657, Loss:

Epoch: 771, Loss: 0.2676957845687866, Accuracy: 89.51799011230469
Epoch: 772, Loss: 0.2676752209663391, Accuracy: 89.51849365234375
Epoch: 773, Loss: 0.26765498518943787, Accuracy: 89.51892852783203
Epoch: 774, Loss: 0.2676346004009247, Accuracy: 89.51962280273438
Epoch: 775, Loss: 0.26761385798454285, Accuracy: 89.52024841308594
Epoch: 776, Loss: 0.26759734749794006, Accuracy: 89.52069091796875
Epoch: 777, Loss: 0.26758018136024475, Accuracy: 89.52144622802734
Epoch: 778, Loss: 0.2675624489784241, Accuracy: 89.52182006835938
Epoch: 779, Loss: 0.26754188537597656, Accuracy: 89.52251434326172
Epoch: 780, Loss: 0.26752346754074097, Accuracy: 89.5230712890625
Epoch: 781, Loss: 0.2675028443336487, Accuracy: 89.52388763427734
Epoch: 782, Loss: 0.26748597621917725, Accuracy: 89.52444458007812
Epoch: 783, Loss: 0.2674679160118103, Accuracy: 89.52494049072266
Epoch: 784, Loss: 0.26744720339775085, Accuracy: 89.52518463134766
Epoch: 785, Loss: 0.2674289345741272, Accuracy: 89.52581024169922
Epo

Epoch: 902, Loss: 0.2655077874660492, Accuracy: 89.5882568359375
Epoch: 903, Loss: 0.2654905617237091, Accuracy: 89.58885955810547
Epoch: 904, Loss: 0.26547208428382874, Accuracy: 89.58934783935547
Epoch: 905, Loss: 0.26545450091362, Accuracy: 89.5899429321289
Epoch: 906, Loss: 0.2654379606246948, Accuracy: 89.59036254882812
Epoch: 907, Loss: 0.2654198408126831, Accuracy: 89.59090423583984
Epoch: 908, Loss: 0.2654130458831787, Accuracy: 89.59161376953125
Epoch: 909, Loss: 0.26540064811706543, Accuracy: 89.59256744384766
Epoch: 910, Loss: 0.265385240316391, Accuracy: 89.5927505493164
Epoch: 911, Loss: 0.2653706669807434, Accuracy: 89.59304809570312
Epoch: 912, Loss: 0.2653539478778839, Accuracy: 89.59387969970703
Epoch: 913, Loss: 0.2653381824493408, Accuracy: 89.59423828125
Epoch: 914, Loss: 0.26532894372940063, Accuracy: 89.59488677978516
Epoch: 915, Loss: 0.26531341671943665, Accuracy: 89.59542846679688
Epoch: 916, Loss: 0.26530537009239197, Accuracy: 89.59607696533203
Epoch: 917, Lo

## 데이터셋 및 학습 파라미터 저장

In [38]:
np.savez_compressed('ch2_dataset.npz', inputs = pts, labels = labels)
# savez : 여러개의 numpy object 저장
# compressed : 압축해서 용량 작게 저장

W_h, b_h = model.d1.get_weights()
W_o, b_o = model.d2.get_weights()

W_h = np.transpose(W_h)
W_o = np.transpose(W_o)
np.savez_compressed('ch2_parameters.npz',
                   W_h = W_h,
                   b_h = b_h,
                   W_o = W_o,
                   b_o = b_o)